In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import numpy as np
import pandas as pd
from collections import Counter 
from sklearn.utils import class_weight
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import sys

In [3]:
training_data = pd.read_csv('data/aol/user-ct-test-collection-05.txt', sep="\t")
training_queries = training_data.Query.dropna()
training_tokens_freq = Counter(training_queries)

testing_data = pd.read_csv('data/aol/user-ct-test-collection-04.txt', sep="\t")
testing_queries = training_data.Query.dropna()
testing_tokens_freq = Counter(training_queries)

In [4]:
# Prepare the dataset
X_train = list(training_tokens_freq.keys())
y_train = [1 if freq >= 200 else 0 for freq in training_tokens_freq.values()]
X_test = list(testing_tokens_freq.keys())
y_test = [1 if freq >= 200 else 0 for freq in testing_tokens_freq.values()]

In [61]:
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))

X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [62]:
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}

# Train the model
model = LogisticRegression(class_weight=class_weights_dict)
# model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

/Users/tahahasan/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight={0: 0.5002161539294387, 1: 1157.0831842576029})

In [69]:
# Evaluate the model
predictions = (model.predict_proba(X_test_vectorized)[:, 1] >= 0.95).astype(int)
print("Accuracy:", accuracy_score(y_test, predictions))
print("F1 Score:", f1_score(y_test, predictions))
confusion_matrix(y_test, predictions)

Accuracy: 0.9971606786851461
F1 Score: 0.2016952836339926


array([[1289482,    3578],
       [     95,     464]])

In [64]:
# from Oracle import Oracle

# filename = 'models/logistic_regression.sav'
# pickle.dump(model, open(filename, 'wb'))

In [65]:
# test = vectorizer.transform(training_queries)
# ppp = (model.predict_proba(test)[:, 1] >= 0.97).astype(int)

In [67]:
print((sys.getsizeof(pickle.dumps(model)) + sys.getsizeof(pickle.dumps(vectorizer))) / 10 ** 6)

1.739335
